In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

# Import meter Data

In [2]:
# Read in the data, it is tab seperated.  Note: Google Co-Lab has trouble reading the file, likely due to some type of encoding issue.
relativePath=os.getcwd()
readFilePath=relativePath+"\\IU_Peak.txt"
df = pd.read_csv(readFilePath, sep="\t")

# Convert to datetime and date string into datetime objects.
df.READ_DT = pd.to_datetime(df.READ_DT)
df.READ_DTM = pd.to_datetime(df.READ_DTM)

# Set the index to READ_DTM to allow pandas to treat the dataframe as a timeseries when plotting, etc.
# df.set_index('READ_DTM',drop=False,inplace=True)
df = df.drop(['CHANNEL_ID', 'READ_5MIN_INT', 'UOM', 'STATUS', 'READ_VERSION'],axis=1)

In [4]:
df.head()

METER_ID    READ_DT  READ_HR  READ_30MIN_INT  READ_15MIN_INT  \
0  4440193 2019-01-11       19               1               2   
1  4440193 2019-01-11       19               2               3   
2  4440193 2019-01-11       19               2               4   
3  4440193 2019-01-11       20               1               1   
4  4440193 2019-01-11       20               1               2   

             READ_DTM  READ_VALUE  
0 2019-01-11 18:30:00   49.781986  
1 2019-01-11 18:45:00   47.690464  
2 2019-01-11 19:00:00   47.265308  
3 2019-01-11 19:15:00   46.851425  
4 2019-01-11 19:30:00   47.333904

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12060907 entries, 0 to 12060906
Data columns (total 7 columns):
METER_ID          object
READ_DT           datetime64[ns]
READ_HR           int64
READ_30MIN_INT    int64
READ_15MIN_INT    int64
READ_DTM          datetime64[ns]
READ_VALUE        float64
dtypes: datetime64[ns](2), float64(1), int64(3), object(1)
memory usage: 644.1+ MB


### Filtering Date for Main meters

In [61]:
# DUKE IDs
duke_ids = ['4440099', '4440104', '4440105', '4440012', '4440073', '4440177']

### Main Meter_id = '4440099'

In [62]:
# Dataframe of DUKE meter IDs - 4440099
df_duke_99 = df[df.iloc[:, 0] == duke_ids[0]].reset_index(drop=True)
print(df_duke_99.shape)
df_duke_99.head()

(34952, 7)


METER_ID    READ_DT  READ_HR  READ_30MIN_INT  READ_15MIN_INT  \
0  4440099 2019-01-11        1               1               1   
1  4440099 2019-01-11        1               1               2   
2  4440099 2019-01-11        1               2               3   
3  4440099 2019-01-11        1               2               4   
4  4440099 2019-01-11        2               1               1   

             READ_DTM   READ_VALUE  
0 2019-01-11 00:15:00  3106.110352  
1 2019-01-11 00:30:00  3080.762207  
2 2019-01-11 00:45:00  3058.536133  
3 2019-01-11 01:00:00  3058.188232  
4 2019-01-11 01:15:00  3039.658936

In [177]:
df_duke_99.describe()

READ_HR  READ_30MIN_INT  READ_15MIN_INT    READ_VALUE  \
count  34952.000000    34952.000000     34952.00000  34952.000000   
mean      12.489757        1.500000         2.50000   2597.204052   
std        6.924685        0.500007         1.11805    639.314675   
min        1.000000        1.000000         1.00000      0.000000   
25%        6.000000        1.000000         1.75000   2343.188416   
50%       12.000000        1.500000         2.50000   2699.616943   
75%       18.000000        2.000000         3.25000   2996.539124   
max       24.000000        2.000000         4.00000   4504.577637   

              MONTH          HOUR           DAY       HOLIDAY  
count  34952.000000  34952.000000  34952.000000  34952.000000  
mean       6.487068     11.489929     15.659504      0.408532  
std        3.447378      6.924189      8.817276      0.491569  
min        1.000000      0.000000      1.000000      0.000000  
25%        4.000000      5.000000      8.000000      0.000000  
50%        7.000000     11.000000     16.000000      0.000000  
75%        9.000000     17.000000     23.000000      1.000000  
max       12.000000     23.000000     31.000000      1.000000

In [63]:
df_duke_99['METER_ID'].value_counts()

4440099    34952
Name: METER_ID, dtype: int64

# Feature Engineering

### DATE split

In [64]:
df_duke_99['MONTH'] = df_duke_99['READ_DTM'].dt.month
df_duke_99['HOUR'] = df_duke_99['READ_DTM'].dt.hour
df_duke_99['DATE'] = df_duke_99['READ_DTM'].dt.date
df_duke_99['DAY'] = df_duke_99['READ_DTM'].dt.day
df_duke_99['WEEKDAY'] = df_duke_99['READ_DTM'].dt.weekday_name

### Academic_Period

In [65]:
df_duke_99['academic_period'] = 'Break'
df_duke_99['academic_period'] = np.where((df_duke_99.READ_DT >= '2019-01-07') & (df_duke_99.READ_DT <= '2019-05-03'),'Spring_2019',df_duke_99.academic_period)
df_duke_99['academic_period'] = np.where((df_duke_99.READ_DT >= '2019-05-07') & (df_duke_99.READ_DT <= '2019-07-26'),'Summer_2019',df_duke_99.academic_period)
df_duke_99['academic_period'] = np.where((df_duke_99.READ_DT >= '2019-08-26') & (df_duke_99.READ_DT <= '2019-12-20'),'Fall_2019',df_duke_99.academic_period)


### Season

In [66]:
df_duke_99['SEASON'] = ''
df_duke_99['SEASON'] = np.where((df_duke_99.READ_DT >= '2019-03-20') & (df_duke_99.READ_DT <= '2019-06-20'),'Spring',df_duke_99.SEASON)
df_duke_99['SEASON'] = np.where((df_duke_99.READ_DT >= '2019-06-21') & (df_duke_99.READ_DT <= '2019-09-22'),'Summer',df_duke_99.SEASON)
df_duke_99['SEASON'] = np.where((df_duke_99.READ_DT >= '2019-09-23') & (df_duke_99.READ_DT <= '2019-12-20'),'Fall',df_duke_99.SEASON)
df_duke_99['SEASON'] = np.where((df_duke_99.READ_DT >= '2019-12-21') & (df_duke_99.READ_DT <= '2019-12-31'),'Winter',df_duke_99.SEASON)
df_duke_99['SEASON'] = np.where((df_duke_99.READ_DT >= '2019-01-01') & (df_duke_99.READ_DT <= '2019-03-19'),'Winter',df_duke_99.SEASON)


### HOLIDAY Flag - Set to 1 for Weekends, Official Holidays, Recess period and for academic period breaks

In [67]:
HOLIDAYS = ['2019-01-20', '2019-05-27', '2019-07-04', '2019-09-02', 
            '2019-03-18', '2019-03-19', '2019-03-20', '2019-03-21', '2019-03-22', 
            '2019-11-25', '2019-11-26', '2019-11-27', '2019-11-28', '2019-11-29']

In [68]:
df_duke_99['HOLIDAY'] = 0
df_duke_99['HOLIDAY'] = np.where((df_duke_99.READ_DT.isin(HOLIDAYS)) | (df_duke_99.academic_period == 'Break')
                                 | (df_duke_99.WEEKDAY.isin(['Saturday', 'Sunday'])), 1, df_duke_99.HOLIDAY)

In [69]:
df_duke_99.head()

METER_ID    READ_DT  READ_HR  READ_30MIN_INT  READ_15MIN_INT  \
0  4440099 2019-01-11        1               1               1   
1  4440099 2019-01-11        1               1               2   
2  4440099 2019-01-11        1               2               3   
3  4440099 2019-01-11        1               2               4   
4  4440099 2019-01-11        2               1               1   

             READ_DTM   READ_VALUE  MONTH  HOUR        DATE  DAY WEEKDAY  \
0 2019-01-11 00:15:00  3106.110352      1     0  2019-01-11   11  Friday   
1 2019-01-11 00:30:00  3080.762207      1     0  2019-01-11   11  Friday   
2 2019-01-11 00:45:00  3058.536133      1     0  2019-01-11   11  Friday   
3 2019-01-11 01:00:00  3058.188232      1     1  2019-01-11   11  Friday   
4 2019-01-11 01:15:00  3039.658936      1     1  2019-01-11   11  Friday   

  academic_period  SEASON  HOLIDAY  
0     Spring_2019  Winter        0  
1     Spring_2019  Winter        0  
2     Spring_2019  Winter        0  
3     Spring_2019  Winter        0  
4     Spring_2019  Winter        0

In [70]:
df_duke_99.shape

(34952, 15)

# Import Weather Data

In [71]:
readFilePath=relativePath+"\\weather_2019_hourly.txt"
w_df = pd.read_csv(readFilePath, sep=",")
w_df.columns = w_df.columns.str.replace(" ","_").str.lower()
w_df.head(3)

location                                            address  \
0  IU Bloomington  107 S Indiana Ave, Bloomington, IN 47405, Unit...   
1  IU Bloomington  107 S Indiana Ave, Bloomington, IN 47405, Unit...   
2  IU Bloomington  107 S Indiana Ave, Bloomington, IN 47405, Unit...   
3  IU Bloomington  107 S Indiana Ave, Bloomington, IN 47405, Unit...   
4  IU Bloomington  107 S Indiana Ave, Bloomington, IN 47405, Unit...   

                                    resolved_address            date_time  \
0  107 S Indiana Ave, Bloomington, IN 47405, Unit...  01/01/2019 00:00:00   
1  107 S Indiana Ave, Bloomington, IN 47405, Unit...  01/01/2019 01:00:00   
2  107 S Indiana Ave, Bloomington, IN 47405, Unit...  01/01/2019 02:00:00   
3  107 S Indiana Ave, Bloomington, IN 47405, Unit...  01/01/2019 03:00:00   
4  107 S Indiana Ave, Bloomington, IN 47405, Unit...  01/01/2019 04:00:00   

   maximum_temperature  minimum_temperature  temperature  wind_chill  \
0                 44.9                 44.9         44.9        37.8   
1                 44.0                 44.0         44.0        37.3   
2                 43.1                 43.1         43.1        37.4   
3                 42.2                 42.2         42.2        36.7   
4                 41.1                 41.1         41.1        35.4   

   heat_index  precipitation  snow_depth  wind_speed  wind_gust  cloud_cover  \
0         NaN            NaN         NaN        16.1       32.2         92.9   
1         NaN            NaN         NaN        13.9        NaN         92.9   
2         NaN            NaN         NaN        10.3        NaN         92.9   
3         NaN            NaN         NaN         9.2        NaN         92.9   
4         NaN            NaN         NaN         9.2        NaN         92.9   

   relative_humidity conditions  \
0              79.47   Overcast   
1              82.24   Overcast   
2              88.16   Overcast   
3              88.20   Overcast   
4              91.81   Overcast   

                               contributing_stations  
0  MONROE COUNTY AIRPORT, (Id=72437503893,5.4mi, ...  
1  MONROE COUNTY AIRPORT, (Id=72437503893,5.4mi, ...  
2  MONROE COUNTY AIRPORT, (Id=72437503893,5.4mi, ...  
3  MONROE COUNTY AIRPORT, (Id=72437503893,5.4mi, ...  
4  MONROE COUNTY AIRPORT, (Id=72437503893,5.4mi, ...

In [72]:
w_df.date_time = pd.to_datetime(w_df.date_time)
w_df['HOUR'] = w_df['date_time'].dt.hour
w_df['DATE'] = w_df['date_time'].dt.date

In [73]:
w_df = w_df.drop(['snow_depth', 'location', 'address', 'resolved_address', 'heat_index', 'wind_gust', 'date_time',
                  'maximum_temperature', 'minimum_temperature', 'contributing_stations', 'cloud_cover'], axis=1)

In [74]:
w_df.head()

temperature  wind_chill  precipitation  wind_speed  relative_humidity  \
0         44.9        37.8            NaN        16.1              79.47   
1         44.0        37.3            NaN        13.9              82.24   
2         43.1        37.4            NaN        10.3              88.16   
3         42.2        36.7            NaN         9.2              88.20   
4         41.1        35.4            NaN         9.2              91.81   

  conditions  HOUR        DATE  
0   Overcast     0  2019-01-01  
1   Overcast     1  2019-01-01  
2   Overcast     2  2019-01-01  
3   Overcast     3  2019-01-01  
4   Overcast     4  2019-01-01

# Merge Duke Meter data with Weather data

In [124]:
df_merge = pd.merge(df_duke_99, w_df,  how='left', left_on=['DATE','HOUR'], right_on = ['DATE','HOUR'])

In [125]:
df_merge.head()

METER_ID    READ_DT  READ_HR  READ_30MIN_INT  READ_15MIN_INT  \
0  4440099 2019-01-11        1               1               1   
1  4440099 2019-01-11        1               1               2   
2  4440099 2019-01-11        1               2               3   
3  4440099 2019-01-11        1               2               4   
4  4440099 2019-01-11        2               1               1   

             READ_DTM   READ_VALUE  MONTH  HOUR        DATE  ...  WEEKDAY  \
0 2019-01-11 00:15:00  3106.110352      1     0  2019-01-11  ...   Friday   
1 2019-01-11 00:30:00  3080.762207      1     0  2019-01-11  ...   Friday   
2 2019-01-11 00:45:00  3058.536133      1     0  2019-01-11  ...   Friday   
3 2019-01-11 01:00:00  3058.188232      1     1  2019-01-11  ...   Friday   
4 2019-01-11 01:15:00  3039.658936      1     1  2019-01-11  ...   Friday   

  academic_period  SEASON HOLIDAY  temperature  wind_chill  precipitation  \
0     Spring_2019  Winter       0         18.2         NaN            NaN   
1     Spring_2019  Winter       0         18.2         NaN            NaN   
2     Spring_2019  Winter       0         18.2         NaN            NaN   
3     Spring_2019  Winter       0         19.3         NaN            NaN   
4     Spring_2019  Winter       0         19.3         NaN            NaN   

   wind_speed  relative_humidity  conditions  
0         NaN              94.88       Clear  
1         NaN              94.88       Clear  
2         NaN              94.88       Clear  
3         NaN              86.94       Clear  
4         NaN              86.94       Clear  

[5 rows x 21 columns]

In [126]:
df_merge = df_merge.drop(['METER_ID', 'READ_DT', 'READ_HR', 'READ_15MIN_INT', 'READ_DTM', 'READ_30MIN_INT', 'DATE'], axis=1)

In [127]:
df_merge.isnull().sum()

READ_VALUE               0
MONTH                    0
HOUR                     0
DAY                      0
WEEKDAY                  0
academic_period          0
SEASON                   0
HOLIDAY                  0
temperature            197
wind_chill           24056
precipitation        33840
wind_speed            7516
relative_humidity     1333
conditions             189
dtype: int64

## Imputing Missing values

In [128]:
df_merge['wind_chill'].fillna(0, inplace=True)
df_merge['precipitation'].fillna(0, inplace=True)
df_merge['relative_humidity'].fillna(0, inplace=True)
df_merge['wind_speed'].fillna(0, inplace=True)
# df_merge['conditions'].fillna('Clear', inplace=True)
# df_merge['temperature'].fillna(df_merge['temperature'].mean(), inplace=True)

In [129]:
df_merge.head()

READ_VALUE  MONTH  HOUR  DAY WEEKDAY academic_period  SEASON  HOLIDAY  \
0  3106.110352      1     0   11  Friday     Spring_2019  Winter        0   
1  3080.762207      1     0   11  Friday     Spring_2019  Winter        0   
2  3058.536133      1     0   11  Friday     Spring_2019  Winter        0   
3  3058.188232      1     1   11  Friday     Spring_2019  Winter        0   
4  3039.658936      1     1   11  Friday     Spring_2019  Winter        0   

   temperature  wind_chill  precipitation  wind_speed  relative_humidity  \
0         18.2         0.0            0.0         0.0              94.88   
1         18.2         0.0            0.0         0.0              94.88   
2         18.2         0.0            0.0         0.0              94.88   
3         19.3         0.0            0.0         0.0              86.94   
4         19.3         0.0            0.0         0.0              86.94   

  conditions  
0      Clear  
1      Clear  
2      Clear  
3      Clear  
4      Clear

In [130]:
df_merge.shape

(34960, 14)

# Modeling

### Split the Datatset

In [131]:
X = df_merge.drop(['READ_VALUE'], axis=1)
y = df_merge["READ_VALUE"]

In [132]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

In [157]:
print(X_train_trans.shape, X_valid_trans.shape, X_test_trans.shape)

(25258, 31) (5244, 31) (4458, 31)


In [133]:
num_features = X_train.select_dtypes(exclude=['object']).columns.tolist()

In [134]:
print(num_features)

['MONTH', 'HOUR', 'DAY', 'HOLIDAY', 'temperature', 'wind_chill', 'precipitation', 'wind_speed', 'relative_humidity']


In [135]:
cat_features = X_train.select_dtypes('object').columns.tolist()

In [136]:
print(cat_features)

['WEEKDAY', 'academic_period', 'SEASON', 'conditions']


### Build Pipeline

In [137]:
# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [141]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Identify the numeric features we wish to consider. 
num_attribs = num_features

# Numerical Features mean imputing
num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', SimpleImputer(strategy='mean')),
        ('std_scaler', StandardScaler()),
    ])

# Identify the categorical features we wish to consider.
cat_attribs = cat_features

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attribs)),
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ohe', OneHotEncoder(sparse=False, handle_unknown="ignore"))
    ])

data_prep_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ], n_jobs=4)

### Fit and Transform Data in the Pipeline

In [142]:
%%time
X_train_trans = data_prep_pipeline.fit_transform(X_train)

Wall time: 4.53 s


In [143]:
X_train_trans.shape

(25258, 31)

In [144]:
X_valid_trans = data_prep_pipeline.transform(X_valid)
X_test_trans = data_prep_pipeline.transform(X_test)

## Baseline Model - Linear Regression

In [146]:
%%time 
np.random.seed(42)

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

estimator_pipeline = Pipeline([
        ("linear", LinearRegression())
    ])

#print(preparation_and_feature_selection_pipeline)
estimator_pipeline.fit(X_train_trans, y_train) 

Wall time: 35.3 ms


Pipeline(memory=None,
         steps=[('linear',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [147]:
y_valid_pred = estimator_pipeline.predict(X_valid_trans)
y_test_pred = estimator_pipeline.predict(X_test_trans)

### Evaluation Metrics - RMSE and MAE

In [148]:
# RMSE
lin_mse = mean_squared_error(y_test, y_test_pred)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

333.87842977094

In [149]:
# MAE
lin_mae = mean_absolute_error(y_test, y_test_pred)
lin_mae

260.16363272393517

In [153]:
y_test_pred

array([2440.00523722, 2568.23443976, 2619.75803594, ..., 2543.0967806 ,
       2751.86026309, 2684.10733279])

## Decision Tree Regressor

In [158]:
from sklearn.tree import DecisionTreeRegressor

estimator_pipeline = Pipeline([
        ("Decision_tree", DecisionTreeRegressor(random_state=42))
    ])

#print(preparation_and_feature_selection_pipeline)
estimator_pipeline.fit(X_train_trans, y_train) 

Pipeline(memory=None,
         steps=[('Decision_tree',
                 DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                       max_depth=None, max_features=None,
                                       max_leaf_nodes=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       presort='deprecated', random_state=42,
                                       splitter='best'))],
         verbose=False)

In [159]:
y_valid_pred = estimator_pipeline.predict(X_valid_trans)
y_test_pred = estimator_pipeline.predict(X_test_trans)

In [160]:
# RMSE
dt_mse = mean_squared_error(y_test, y_test_pred)
dt_rmse = np.sqrt(dt_mse)
dt_rmse

46.68603408353917

In [161]:
# MAE
dt_mae = mean_absolute_error(y_test, y_test_pred)
dt_mae

24.913090422507143

In [162]:
y_test_pred

array([2192.98714193, 3040.6652832 , 2666.47753906, ..., 2371.00813802,
       2892.90673828, 2564.6036377 ])

## SVM Regressor

In [163]:
from sklearn.svm import SVR

estimator_pipeline = Pipeline([
        ("SVR", SVR(kernel="linear"))
    ])

#print(preparation_and_feature_selection_pipeline)
estimator_pipeline.fit(X_train_trans, y_train) 

Pipeline(memory=None,
         steps=[('SVR',
                 SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
                     gamma='scale', kernel='linear', max_iter=-1,
                     shrinking=True, tol=0.001, verbose=False))],
         verbose=False)

In [164]:
y_valid_pred = estimator_pipeline.predict(X_valid_trans)
y_test_pred = estimator_pipeline.predict(X_test_trans)

In [173]:
# RMSE
svr_mse = mean_squared_error(y_test, y_test_pred)
svr_rmse = np.sqrt(svr_mse)
svr_rmse

334.4028076333028

In [166]:
# MAE
svr_mae = mean_absolute_error(y_test, y_test_pred)
svr_mae

267.35268853167986

In [167]:
y_test_pred

array([2437.08824682, 2549.82889633, 2635.68063991, ..., 2514.67064601,
       2769.03606561, 2648.89668454])

## SGD Regressor

In [168]:
from sklearn import linear_model

estimator_pipeline = Pipeline([
        ("SGD", linear_model.SGDRegressor(max_iter=1000, tol=1e-3))
    ])

#print(preparation_and_feature_selection_pipeline)
estimator_pipeline.fit(X_train_trans, y_train) 

Pipeline(memory=None,
         steps=[('SGD',
                 SGDRegressor(alpha=0.0001, average=False, early_stopping=False,
                              epsilon=0.1, eta0=0.01, fit_intercept=True,
                              l1_ratio=0.15, learning_rate='invscaling',
                              loss='squared_loss', max_iter=1000,
                              n_iter_no_change=5, penalty='l2', power_t=0.25,
                              random_state=None, shuffle=True, tol=0.001,
                              validation_fraction=0.1, verbose=0,
                              warm_start=False))],
         verbose=False)

In [169]:
y_valid_pred = estimator_pipeline.predict(X_valid_trans)
y_test_pred = estimator_pipeline.predict(X_test_trans)

In [174]:
# RMSE
sgd_mse = mean_squared_error(y_test, y_test_pred)
sgd_rmse = np.sqrt(sgd_mse)
sgd_rmse

334.4028076333028

In [175]:
# MAE
sgd_mae = mean_absolute_error(y_test, y_test_pred)
sgd_mae

260.7040087073004

In [172]:
y_test_pred

array([2431.41119007, 2546.49715044, 2615.05364592, ..., 2540.32739109,
       2748.85596438, 2689.82978635])